# Fixed bed model example

Jialu Wang (jwang44@nd.edu) and Alex Dowling (adowling@nd.edu), University of Notre Dame

This notebook conducts design of experiments for the fixed bed CO$_2$ adsorption experiment. The DoE package is used and the DoE results are shown by heatmaps of three optimalities, A-, D- and E-optimalities. 

## Import Model

In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd

In [4]:
import fixed_bed_model_indexed_dynamic as mod
from fim_doe import * 

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)
    'pyomo.common.sorting.sorted_robust'.  Please update your import.
    (deprecated in 6.1) (called from <frozen importlib._bootstrap>:228)


/Users/wangjialu/pyomo/pyomo/contrib/doe/fixed_bed_model_indexed_dynamic.py:289: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if diff is 0:
/Users/wangjialu/pyomo/pyomo/contrib/doe/fixed_bed_model_indexed_dynamic.py:291: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif diff is 1:
/Users/wangjialu/pyomo/pyomo/contrib/doe/fixed_bed_model_indexed_dynamic.py:293: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif diff is -1:
/Users/wangjialu/pyomo/pyomo/contrib/doe/fixed_bed_model_indexed_dynamic.py:295: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif diff is 2:
/Users/wangjialu/pyomo/pyomo/contrib/doe/fixed_bed_model_indexed_dynamic.py:305: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if i is 'base' or 'forward_ua' or 'backward_ua':
/Users/wangjialu/pyomo/pyomo/contrib/doe/fixed_bed_model_indexed_dynamic.py:307: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif i is 'forward_k':
/Users/wangjialu/pyomo/pyomo/contrib/

### Define Fixed Bed Model

In [5]:
# Define the fixed bed model timepoints 
NFEt=68
#NFEt = 102
time_scale=3200
#time_scale = 4800
collo=False

# a list of timepoints for creating the model
exp_time = mod.time_points(NFEt, time_scale)

In [6]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    mod.add_variables(doe, timesteps=timepoints, start=time_start)
    
    mod.add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    mod.initial_bed_csv(doe, store)
    mod.fix_initial_bed(doe)

In [7]:
def fixed_bed_run(dv=[313.15, 0.15], param_init = [212, np.log(5.0E6)], init_point='20200729_energy.csv', energy_opt=True):
    '''
    Create model function of fixed bed model specifically 
    
    init_point: initial solutions
    energy_opt: if the energy balance is toggled on
    '''
    test = mod.create_model(temp=dv[0], temp_bath=313.15, y=dv[1], para=param_init[0], ua=param_init[1], opt=True, conti=False, k_aug=False, diff=0, energy=energy_opt, doe_model=True)
    add_model(test, exp_time)
    discretize(test, no_points=NFEt)
    initialize(test, init_point)
    return test

In [8]:
def fixed_bed_isothermal(dv=[313.15, 0.15], param_init = [212], init_point='20200729_energy.csv', energy_opt=False):
    '''
    Create model function of fixed bed model specifically 
    
    init_point: initial solutions
    energy_opt: if the energy balance is toggled on
    '''
    test = mod.create_model(temp=dv[0], temp_bath=313.15, y=dv[1], para=param_init[0], ua=np.log(5.0E6), opt=True, conti=False, k_aug=False, diff=0, energy=energy_opt, doe_model=True)
    add_model(test, exp_time)
    discretize(test, no_points=NFEt)
    initialize(test, init_point)
    return test

## Defines Problem 



In [9]:
T_range = [293.15]
y_range = [0.4]
#T_range = [293.15, 303.15, 313.15, 323.15, 333.15, 343.15, 353.15, 363.16, 373.15]
#T_range = [293, 303, 313, 323,333, 343, 353, 363, 373]
#y_range = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]

print(T_range)
print(y_range)

param_initial_re = {'fitted_transport_coefficient': 212 ,
                    'ua': np.log(5.0E3)}

design_ranges_re = {'temp_orig': T_range,
                    'yfeed': y_range}

[293.15]
[0.4]


In [10]:
def cut_jac(jaco_sam, add=0):
    if add==0:
        jaco=np.zeros((2,103))
        jaco[0] = jaco_sam['fitted_transport_coefficient'].iloc[:103]
        jaco[1] = jaco_sam['ua'][:103]
    if add==1:
        jaco=np.zeros((2,206))
        jaco[0] = jaco_sam['fitted_transport_coefficient'][:206]
        jaco[1] = jaco_sam['ua'][:206]
    elif add==2:
        jaco=np.zeros((2,206))
        jaco[0][:103] = jaco_sam['fitted_transport_coefficient'][:103]
        jaco[1][:103] = jaco_sam['ua'][:103]
        jaco[0][103:206] = jaco_sam['fitted_transport_coefficient'][206:309]
        jaco[1][103:206] = jaco_sam['ua'][206:309]

    return jaco

def cutted_fim(name, option):
    kitty = pd.read_csv(name)
    print(kitty)
    kitty_cut = cut_jac(kitty, add=option)
    jaco_store = pd.DataFrame({'A1': kitty_cut[0],
                              'A2': kitty_cut[1]})
    fim = jaco_store.T@jaco_store
    return fim
    
    

In [11]:
prior_293_40 = cutted_fim('/Users/wangjialu/Desktop/final-result-fixed-bed/fixed_bed/jacobian_293_40_102_5e6.csv', 0)
prior_373_40 = cutted_fim('/Users/wangjialu/Desktop/final-result-fixed-bed/fixed_bed/jacobian_373_40_102_5e6.csv',0) 

prior_all = prior_293_40 + prior_373_40 
prior_pass=prior_all.to_numpy()

print('The prior information FIM:', prior_pass)
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvector of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

     Unnamed: 0  temp_orig  yfeed  fitted_transport_coefficient            ua
0             0          0      0                  0.000000e+00  0.000000e+00
1             1          0      0                  5.093802e-06 -3.181793e-08
2             2          0      0                  2.859918e-05 -1.018305e-07
3             3          0      0                  1.209566e-04 -5.349883e-07
4             4          0      0                  5.350392e-04 -2.378592e-06
..          ...        ...    ...                           ...           ...
304         304          0      0                  8.010630e-07 -1.731524e-06
305         305          0      0                  7.037443e-07 -1.501550e-06
306         306          0      0                  6.181864e-07 -1.302124e-06
307         307          0      0                  5.429882e-07 -1.129186e-06
308         308          0      0                  4.769987e-07 -9.791961e-07

[309 rows x 5 columns]
     Unnamed: 0  temp_orig  yfeed  fitte

In [12]:
prior_293_40 = pd.read_csv('/Users/wangjialu/Desktop/final-result-fixed-bed/fixed_bed/fim_293_40_scale_5e6.csv')
prior_373_40 = pd.read_csv('/Users/wangjialu/Desktop/final-result-fixed-bed/fixed_bed/fim_373_40_scale_5e6.csv') 

prior_all = prior_293_40 
prior_pass=prior_all.to_numpy()

print('The prior information FIM:', prior_pass)
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvector of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[ 2.17896149e-02 -1.40959783e-04]
 [-1.40959783e-04  9.20117406e-07]]
Eigenvalue of the prior experiments FIM: [2.17905267e-02 8.23034696e-09]
Eigenvector of the prior experiments FIM: [[-0.00646899 -0.99997908]
 [-0.99997908  0.00646899]]


## Compute FIM

In [ ]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

# Define experiments


## Perform sequential sensitivity analysis

In [ ]:
# Decide if pass any prior information
#exist_info=prior_pass
exist_info = None

# Decide if scaling 
scale_option=True

save_option='total'

In [ ]:
# create the object
searcher = DesignOfExperiments(fixed_bed_isothermal, mod.extract_fixed, design_ranges_re, param_initial_re, prior_FIM=exist_info)


In [ ]:
# solve the doe grid search problem 
jaco_in, fim_info = searcher.run_grid_search(scale=scale_option, save_jac=save_option)


In [ ]:
print(fim_info)

### Generate prior info

In [ ]:
def prior_generate(path, names):
    jaco1 = pd.read_csv(path)
    #jaco1 = jaco[(jaco[self.dv[0]]==0) & (jaco[self.dv[1]]==0)]
    jaco_array = []
    for name in list(param_initial_re.keys()):
        jaco_array.append(list(jaco1[name]))

    jaco_store = pd.DataFrame({'A1': jaco_array[0],
                              'A2': jaco_array[1]})
    #print(jaco_store)
    #jaco_array = np.asarray(jaco_array)


    fim = jaco_store.T@jaco_store  
    fim.to_csv(names,index=False)
    
prior_generate('jacobian_373_40_102_5e6.csv', 'fim_373_40_102_5e6.csv')

## Load from Jacobian files

In [ ]:
# for only k

prior_293_40 = [0.02034514]
prior_293_10 = [0.00078969]
prior_373_10 = [0.0001321]
prior_373_40 = [0.00106497]

prior_pass = [prior_373_40[0] + prior_293_40[0]]
print(prior_pass)

In [ ]:
# Decide if pass any prior information
exist_info=prior_pass
#exist_info = None

# Decide if scaling 
scale_option=True

save_option=None

In [ ]:
def cutting(jaco_sam, add=0):
    if add==0:
        jaco=np.zeros((2,103))
        jaco[0] = jaco_sam[0][:103]
        jaco[1] = jaco_sam[1][:103]
    if add==1:
        jaco=np.zeros((2,206))
        jaco[0] = jaco_sam[0][:206]
        jaco[1] = jaco_sam[1][:206]
    elif add==2:
        jaco=np.zeros((2,206))
        jaco[0][:103] = jaco_sam[0][:103]
        jaco[1][:103] = jaco_sam[1][:103]
        jaco[0][103:206] = jaco_sam[0][206:309]
        jaco[1][103:206] = jaco_sam[1][206:309]

    return jaco

def cutting_konly(jaco_sam):
    jaco=np.zeros((1,103))
    jaco[0] = jaco_sam[0][:103]
        
    return jaco

In [ ]:
# First define the searcher object
searcher = DesignOfExperiments(fixed_bed_run, mod.extract_fixed, design_ranges_re, param_initial_re, prior_FIM=exist_info)

jaco_in, fim_in, dict_jaco = searcher.read_csv(file='/Users/wangjialu/Desktop/final-result/fixed_bed/jacobian_scale_5e6_102.csv', 
                                    visual=True, cut=True, cut_func=cutting)


In [ ]:
print(fim_in)

prior_293_40 = fim_in[(0,6)]
prior_293_10 = fim_in[(0,0)]
prior_373_10 = fim_in[(8,0)]
prior_373_40 = fim_in[(8,6)]

print(prior_293_40)
print(prior_293_10)
print(prior_373_10)
print(prior_373_40)

## Visualize DoE results

In [ ]:
jaco_in.calculate_criteria(save=True)
jaco_in.heatmap('DoE for Fixed Bed Model', 'Bed temperature [K]', 'CO\N{SUBSCRIPT TWO} feed composition [mol %]')